In [1]:
import datasets

In [2]:
datasets.__version__

'3.5.0'

In [3]:
from datasets import load_dataset

gtzan = load_dataset("marsyas/gtzan", 'all')
gtzan

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 999
    })
})

In [4]:
gtzan = gtzan['train'].train_test_split(seed=42, shuffle=True, test_size = 0.1)
gtzan

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 899
    })
    test: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 100
    })
})

In [5]:
gtzan['train'][0]

{'file': '/teamspace/studios/this_studio/.cache/huggingface/datasets/downloads/extracted/bc77c085c903abd055411ef5d24a7df6b0b7223c011f882a8054ccedd5b788d3/genres/pop/pop.00098.wav',
 'audio': {'path': '/teamspace/studios/this_studio/.cache/huggingface/datasets/downloads/extracted/bc77c085c903abd055411ef5d24a7df6b0b7223c011f882a8054ccedd5b788d3/genres/pop/pop.00098.wav',
  'array': array([ 0.10720825,  0.16122437,  0.28585815, ..., -0.22924805,
         -0.20629883, -0.11334229]),
  'sampling_rate': 22050},
 'genre': 7}

In [6]:
id2label_fn = gtzan['train'].features['genre'].int2str
id2label_fn(gtzan['train'][0]['genre'])

'pop'

In [11]:
import gradio as gr


def generate_audio():
    example = gtzan["train"].shuffle()[0]
    audio = example["audio"]
    return (
        audio["sampling_rate"],
        audio["array"],
    ), id2label_fn(example["genre"])


with gr.Blocks() as demo:
    with gr.Column():
        for _ in range(4):
            audio, label = generate_audio()
            output = gr.Audio(audio, label=label)

demo.launch(debug=True,share=True)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/gradio/processing_utils.py:777: UserWarning: Trying to convert audio automatically from float64 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://d76dc4b6cebf13d59b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://d76dc4b6cebf13d59b.gradio.live


## Fine tune a genre classification model

In [7]:
from transformers import AutoFeatureExtractor

In [8]:
model_id = "ntu-spml/distilhubert"
feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_id, do_normalize=True, return_attention_mask=True
)

In [9]:
sampling_rate = feature_extractor.sampling_rate
sampling_rate

16000

Resampling using `cast_column`

In [10]:
from datasets import Audio
gtzan = gtzan.cast_column("audio", Audio(sampling_rate=sampling_rate))

In [11]:
gtzan["train"][0]

{'file': '/teamspace/studios/this_studio/.cache/huggingface/datasets/downloads/extracted/bc77c085c903abd055411ef5d24a7df6b0b7223c011f882a8054ccedd5b788d3/genres/pop/pop.00098.wav',
 'audio': {'path': '/teamspace/studios/this_studio/.cache/huggingface/datasets/downloads/extracted/bc77c085c903abd055411ef5d24a7df6b0b7223c011f882a8054ccedd5b788d3/genres/pop/pop.00098.wav',
  'array': array([ 0.0873509 ,  0.20183384,  0.4790867 , ..., -0.18743178,
         -0.23294401, -0.13517427]),
  'sampling_rate': 16000},
 'genre': 7}

Understand the normalization of feature_extractor

In [12]:
import numpy as np

sample = gtzan["train"][0]["audio"]

print(f"Mean: {np.mean(sample['array']):.3}, Variance: {np.var(sample['array']):.3}")


Mean: 0.000185, Variance: 0.0493


In [13]:
inputs = feature_extractor(sample["array"], sampling_rate=sample["sampling_rate"])

print(f"inputs keys: {list(inputs.keys())}")

print(
    f"Mean: {np.mean(inputs['input_values']):.3}, Variance: {np.var(inputs['input_values']):.3}"
)

inputs keys: ['input_values', 'attention_mask']
Mean: -7.45e-09, Variance: 1.0


In [14]:
max_duration = 30.0


def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
        return_attention_mask=True,
    )
    return inputs

In [15]:
gtzan_encoded = gtzan.map(
    preprocess_function,
    remove_columns=["audio", "file"],
    batched=True,
    batch_size=100,
    num_proc=1,
)
gtzan_encoded

DatasetDict({
    train: Dataset({
        features: ['genre', 'input_values', 'attention_mask'],
        num_rows: 899
    })
    test: Dataset({
        features: ['genre', 'input_values', 'attention_mask'],
        num_rows: 100
    })
})

In [16]:
gtzan_encoded = gtzan_encoded.rename_column("genre", "label")


In [17]:
id2label = {
    str(i): id2label_fn(i)
    for i in range(len(gtzan_encoded["train"].features["label"].names))
}
label2id = {v: k for k, v in id2label.items()}

id2label["7"]

'pop'

In [18]:
from transformers import AutoModelForAudioClassification

num_labels = len(id2label)

model = AutoModelForAudioClassification.from_pretrained(
    model_id,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at ntu-spml/distilhubert and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
# from huggingface_hub import notebook_login

# notebook_login()

# Start the training

In [20]:
import transformers
print(transformers.__version__)

4.53.2


In [21]:
import transformers
from transformers import TrainingArguments

print(TrainingArguments.__module__)

transformers.training_args


In [22]:
import transformers
print(transformers.__version__)
print(transformers.__file__)

4.53.2
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/__init__.py


In [23]:
!pip install -U "accelerate>=0.26.0"

In [24]:
!pip show transformers
!pip show accelerate


Name: transformers
Version: 4.53.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /system/conda/miniconda3/envs/cloudspace/lib/python3.10/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 
Name: accelerate
Version: 1.9.0
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: /system/conda/miniconda3/envs/cloudspace/lib/python3.10/site-packages
Requires: huggingface_hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: 


In [46]:
!pip install "transformers[torch]"

In [33]:
from transformers import TrainingArguments

model_name = model_id.split("/")[-1]
batch_size = 8
gradient_accumulation_steps = 1
num_train_epochs = 10

training_args = TrainingArguments(
    f"{model_name}-finetuned-gtzan",
    eval_strategy='epoch', # parameter changed name since 4.46
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    # push_to_hub=True,
)

In [34]:
# !pip install evaluate

In [35]:
import evaluate
import numpy as np

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [37]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=gtzan_encoded["train"],
    eval_dataset=gtzan_encoded["test"],
    processing_class=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.908700,1.831543,0.500000
2,1.176500,1.253103,0.620000
3,1.092400,0.983995,0.710000
4,0.618600,0.767269,0.740000
5,0.540700,0.696255,0.820000
6,0.363800,0.605634,0.800000
7,0.233300,0.601350,0.800000
8,0.134100,0.668314,0.800000
9,0.204600,0.680554,0.800000
10,0.088200,0.674422,0.820000


TrainOutput(global_step=1130, training_loss=0.7463544589492072, metrics={'train_runtime': 5728.2784, 'train_samples_per_second': 1.569, 'train_steps_per_second': 0.197, 'total_flos': 6.133988274624e+17, 'train_loss': 0.7463544589492072, 'epoch': 10.0})